In [3]:
#imports
from fugashi import Tagger
import pandas as pd
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer
import unicodedata
from datasets import load_dataset
import itertools

import os
import re
import numpy as np 
from sklearn.metrics import accuracy_score

import transformers

import torch
from torch import cuda
from tqdm import tqdm_notebook as tqdm
device = 'cuda' if cuda.is_available() else 'cpu'

device

ModuleNotFoundError: No module named 'datasets'

In [ ]:
!pip install datasets

# The Data

Converting test, train and val datasets which include kanji to be purely kana (furigana)

In [4]:
#data_files = {"train": "train_split.csv", "test": "test_split.csv", 'val': 'val_split.csv'}
dataset = load_dataset('universal_dependencies', 'ja_gsd')
train_split = dataset['train']
test_split = dataset['test']
val_split = dataset['validation']

train_split_df = pd.DataFrame(data=train_split, columns=train_split.features)
train_split_df = train_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
train_split_df

test_split_df = pd.DataFrame(data=test_split, columns=test_split.features)
test_split_df = test_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
test_split_df

val_split_df = pd.DataFrame(data=val_split, columns=val_split.features)
val_split_df = val_split_df.drop(['lemmas','upos','xpos','feats','head','deprel','deps','misc'], axis=1)
val_split_df

NameError: name 'load_dataset' is not defined

In [5]:
print(len(train_split))
print(len(test_split))
print(len(val_split))
#train_split['text']

NameError: name 'train_split' is not defined

In [4]:
#load kanji data
kanji_df = pd.read_json("kanji.json")
kanji_df = kanji_df.drop(['strokes', 'grade', 'freq', 'jlpt_old', 'jlpt_new', 'meanings', 'wk_radicals', 'wk_readings_kun', 'wk_readings_on', 'wk_meanings', 'wk_level'], axis=0)

In [5]:
# As part of data collection, we need a dataset that is ONLY kana. 
# We could not find a dataset that had both kanji AND kana only version,
# so we took a dataset and converted the kanji to kana. 
# kanji to kana is less ambiguous with how on vs kun readings work, but there is still some abiguity.
# we recognize that here, in the dataset.
def unkanjify(df):
    new_sent = []
    for word in df['tokens']:
        #iterating list of chararacters
        for char in word:
            if char in kanji_df.columns:#is kanji
                if char in kanji_df.columns or char in kanji_df.columns:
                    if kanji_df[char].loc['readings_on']:
                        new_sent.append(kanji_df[char].loc['readings_on'][0])
                    else:
                        new_sent.append(char)
                #kun reading
                else:
                    for c in kanji_df[char].loc['readings_kun'][0]:
                        if c == '.':
                            continue
                        new_sent.append(c)
            else:#is not kanji
                new_sent.append(char)
    joined_sent = ''.join(new_sent)
    return joined_sent

# Applying the function to all splits (train, test and val)
train_split_df['kana'] = train_split_df.apply(lambda x: unkanjify(x), axis=1) 
test_split_df['kana'] = test_split_df.apply(lambda x: unkanjify(x), axis=1) 
val_split_df['kana'] = val_split_df.apply(lambda x: unkanjify(x), axis=1) 
train_split_df

,idx,text,tokens,kana
0,train-s1,ホッケーにはデンジャラスプレーの反則があるので、膝より上にボールを浮かすことは基本的に反則に...,"[ホッケー, に, は, デンジャラス, プレー, の, 反則, が, ある, の, で, ...",ホッケーにはデンジャラスプレーのはんそくがあるので、しつよりじょうにボールをふかすことはきほ...
1,train-s2,また行きたい、そんな気持ちにさせてくれるお店です。,"[また, 行き, たい, 、, そんな, 気持ち, に, さ, せ, て, くれる, お, ...",またこうきたい、そんなきじちにさせてくれるおてんです。
2,train-s3,手に持った特殊な刃物を使ったアクロバティックな体術や、揚羽と薄羽同様にクナイや忍具を使って攻...,"[手, に, 持っ, た, 特殊, な, 刃物, を, 使っ, た, アクロバティック, な...",しゅにじったとくしゅなじんぶつをしったアクロバティックなたいじゅつや、よううとはくうどうよう...
3,train-s4,"3年次にはトータルオフェンスで2,892ヤードを獲得し、これは大学記録となった。","[3, 年次, に, は, トータル, オフェンス, で, 2, ,, 892, ヤード, ...","3ねんじにはトータルオフェンスで2,892ヤードをかくとくし、これはだいがくきろくとなった。"
4,train-s5,葬儀の最中ですよ!,"[葬儀, の, 最中, です, よ, !]",そうぎのさいちゅうですよ!
...,...,...,...,...
7022,train-s7160,「さくらちゃん」と呼ばれている。,"[「, さくら, ちゃん, 」, と, 呼ば, れ, て, いる, 。]",「さくらちゃん」とこばれている。
7023,train-s7161,シェルマルケ氏は新憲法などをめぐり、アハメド暫定大統領と対立していた。,"[シェルマルケ, 氏, は, 新, 憲法, など, を, めぐり, 、, アハメド, 暫定,...",シェルマルケしはしんけんほうなどをめぐり、アハメドざんていだいとうりょうとたいりつしていた。
7024,train-s7162,"自らがオウム真理教ではない別のカルト団体に12年間所属していた経験をもとに,こう語りました。","[自ら, が, オウム, 真理, 教, で, は, ない, 別, の, カルト, 団体, に...",じらがオウムしんりきょうではないべつのカルトだんたいに12ねんかんしょぞくしていたけいけんを...
7025,train-s7163,紅い髪と同じ色の瞳という容姿に加え、眉と鎖骨の下の部分に二つずつ紅い球体が埋まっているという...,"[紅い, 髪, と, 同じ, 色, の, 瞳, と, いう, 容姿, に, 加え, 、, 眉...",こういはつとどうじしょくのどうというようしにかえ、びとさこつのかのぶぶんににつずつこういきゅ...


# Splitting the Data

In [ ]:
#kana is Y
#text is X

tokoenie kona
kona_tokeniz = bert.tokenizer('ホッケーにはデンジャラスプレーの反則があるので、膝より上にボールを浮かすことは基本的に反則に')
input_ids: [[101, 2040, 2001, 4116, 9762, 1029, 102, 4938, 3487, 9762, 2124, 2004, 4116, 9762, 2003, 1037, 7214, 2839, 1999, 14936, 102]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



tokenize konji
bert.tokeinizer('ホッケーにはデンジャラスプレーのはんそくがあるので、しつよりじょうにボールをふかすことはきほ.')
input_ids: [[101, 2040, 4821, 3849, 4831, 5331, 2382, 4938, 34313, 9762, 2124, 2004, 3491, 37183, 2003, 1037, 38193, 2839, 1838, 3781, 102]]
attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]




randomly replace kona(orignal sentance) tokenize with konji('output setnacne') tokens (do ~15%)
                          V                 v					                     v						v	
input_ids: [[101, 2040, 4821, 4116, 9762, 1029, 102, 4938, 3487, 9762, 2124, 2004, 4116, 9762, 2003, 1037, 7214, 2839, 2839, 14936, 102]]
attention_mask: [[1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]]



done!
train your model and predict on testing data


take the length of list, then take numpy random indexes

In [1]:
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

NameError: name 'AutoTokenizer' is not defined

In [2]:
inputsKanji = tokenizer(train_split_df['tokens'],max_length=100,truncation=True,padding='max_length',return_tensors='tf')
inputsKana = tokenizer(train_split_df['kana'],max_length=100,truncation=True,padding='max_length',return_tensors='tf')

NameError: name 'tokenizer' is not defined

In [ ]:
inputsKana['labels'] = inputsKanji['input_ids']

In [ ]:
inp_ids = []
lbs = []
idx = 0
for inp in inputsKana.input_ids.numpy():
    actual_tokens = list(set(range(100)) - 
                         set(np.where((inp == 101) | (inp == 102) 
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens), 
                                     size=num_of_token_to_mask, 
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
inp_ids = tf.convert_to_tensor(inp_ids)
inputsKana['input_ids'] = inp_ids

In [7]:
class MultiLabelDataset(torch.utils.data.Dataset):

    def __init__(self, text, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = text
        self.targets = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

class BERTClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        super(BERTClass, self).__init__()
                   
        self.l1 = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")#BertModel.from_pretrained("bert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 32)
        self.classifier = torch.nn.Linear(32, NUM_OUT)
        self.dropout = torch.nn.Dropout(0.5)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        output = self.classifier(pooler)
#         pooler = torch.nn.Tanh()(pooler)
#         pooler = self.dropout(pooler)
        
        output = self.softmax(output)
        return output
    
def loss_fn(outputs, targets):
    return torch.nn.CrossEntropyLoss()(outputs, targets)

def train(model, training_loader, optimizer):
    model.train()
    for data in tqdm(training_loader):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss
    
def validation(model, testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for data in tqdm(testing_loader):
            targets = data['targets']
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids)
            outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs)
            fin_targets.extend(targets)
    return torch.stack(fin_outputs), torch.stack(fin_targets)

In [8]:
#Vars
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
NUM_OUT = 7 # binary task
LEARNING_RATE = 2e-05

training_data = MultiLabelDataset(train_X, train_y, tokenizer, MAX_LEN)
test_data = MultiLabelDataset(test_X, test_y, tokenizer, MAX_LEN)

train_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }    

training_loader = torch.utils.data.DataLoader(training_data, **train_params)
testing_loader = torch.utils.data.DataLoader(test_data, **test_params)

NameError: name 'train_X' is not defined

In [ ]:
model = BERTClass(NUM_OUT)
model.to(device)    

optimizer_bert = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)